## 0. Libarary Import

> ImportError: libGL.so.1: cannot open shared object file: No such file or directory

1. 커맨드라인에 다음 명령어 실행 `apt-get update`
2. 커맨드라인에 다음 명령어 실행 `apt-get -y install libgl1-mesa-glx`
3. 노트북 커널 재시작

In [2]:
import io
import os
import yaml

import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F

from efficientnet_pytorch import EfficientNet


import albumentations
import albumentations.pytorch


import ipywidgets as widgets
from IPython.display import Image as display_image

from typing import Tuple


## 1. 모델 정의 & 설정
### 사전에 학습된 모델을 로딩

In [3]:
with open("config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
class MyEfficientNet(nn.Module) :
    '''
    EfiicientNet-b4의 출력층만 변경합니다.
    한번에 18개의 Class를 예측하는 형태의 Model입니다.
    '''
    def __init__(self, num_classes: int = 18) :
        super(MyEfficientNet, self).__init__()
        self.EFF = EfficientNet.from_pretrained('efficientnet-b4', in_channels=3, num_classes=num_classes)
    
    def forward(self, x) -> torch.Tensor:
        x = self.EFF(x)
        x = F.softmax(x, dim=1)
        return x

In [5]:
def transform_image(image_bytes) -> torch.Tensor:
    transform = albumentations.Compose([
            albumentations.Resize(height=512, width=384),
            albumentations.Normalize(mean=(0.5, 0.5, 0.5), 
                                     std=(0.2, 0.2, 0.2)),
            albumentations.pytorch.transforms.ToTensorV2()
        ])
    image = Image.open(io.BytesIO(image_bytes))
    image = image.convert('RGB')
    image_array = np.array(image)
    return transform(image=image_array)['image'].unsqueeze(0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyEfficientNet(num_classes=18).to(device)
model.load_state_dict(torch.load(config['model_path'], map_location=device))
model.eval()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100.0%


Loaded pretrained weights for efficientnet-b4


MyEfficientNet(
  (EFF): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (

In [7]:
def get_prediction(image_bytes: bytes) -> Tuple[torch.Tensor, torch.Tensor]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tensor = transform_image(image_bytes=image_bytes).to(device)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    return tensor, y_hat

## 2. Inference 

In [8]:
# ipywidget으로 파일 업로더 생성
uploader = widgets.FileUpload(
    accept='.png, .jpg, .jpeg', 
    multiple=False  
)
display(uploader)


FileUpload(value={}, accept='.png, .jpg, .jpeg', description='Upload')

In [9]:
display_button = widgets.Button(description='Display Image')
display_image_space = widgets.Image()


def on_click_callback(clicked_button: widgets.Button) -> None:
    global content
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    display_image_space.value = content

display_button.on_click(on_click_callback)

display(display_button, display_image_space)
# 처음엔 이미지 넣지 않아서 깨진 표시가 나오지만, 위에 Upload하고 누르면 이미지가 보임

Button(description='Display Image', style=ButtonStyle())

Image(value=b'')

In [10]:
inference_button = widgets.Button(description='Inference!')

inference_output = widgets.Output(layout={'border': '1px solid black'})

def on_click_callback(clicked_button: widgets.Button) -> None:
    with inference_output:
        inference_output.clear_output()
        tensor, output = get_prediction(content)

        print(config['classes'][output.item()])

inference_button.on_click(on_click_callback)

display(inference_button, inference_output)


Button(description='Inference!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))